In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [4]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str

In [5]:
def create_outline(state: BlogState) -> BlogState:
    title = state['title']

    prompt = f'Generate a detailed outline for a blog on topic - {title}'
    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [6]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content
    state['content'] = content

    return state

In [7]:
graph = StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)

workflow = graph.compile()

In [8]:
initial_state = {'title': 'Rise of AI'}
final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI', 'outline': '## Blog Post Outline: Rise of AI: A Comprehensive Look\n\n**I. Introduction (Hook & Context)**\n\n*   **A. Hook:** Start with a captivating statistic, a thought-provoking question, or a real-world anecdote showcasing the impact of AI.\n    *   *Examples:*\n        *   "Did you know AI is projected to contribute \\$15.7 trillion to the global economy by 2030?"\n        *   "Imagine a world where diseases are diagnosed before symptoms even appear. AI is making that a reality."\n        *   "Remember the last time you received a personalized product recommendation online? That\'s AI at work."\n*   **B. Define Artificial Intelligence:** Clearly and concisely explain what AI is.\n    *   *Focus on:*\n        *   The ability of machines to mimic human intelligence.\n        *   Learning, problem-solving, and decision-making capabilities.\n        *   Different levels of AI (Narrow/Weak AI, General/Strong AI, Super AI - briefly mention).\n*   **C. Brief His

In [9]:
print(final_state['outline'])

## Blog Post Outline: Rise of AI: A Comprehensive Look

**I. Introduction (Hook & Context)**

*   **A. Hook:** Start with a captivating statistic, a thought-provoking question, or a real-world anecdote showcasing the impact of AI.
    *   *Examples:*
        *   "Did you know AI is projected to contribute \$15.7 trillion to the global economy by 2030?"
        *   "Imagine a world where diseases are diagnosed before symptoms even appear. AI is making that a reality."
        *   "Remember the last time you received a personalized product recommendation online? That's AI at work."
*   **B. Define Artificial Intelligence:** Clearly and concisely explain what AI is.
    *   *Focus on:*
        *   The ability of machines to mimic human intelligence.
        *   Learning, problem-solving, and decision-making capabilities.
        *   Different levels of AI (Narrow/Weak AI, General/Strong AI, Super AI - briefly mention).
*   **C. Brief History of AI:** A quick overview of key milestones in 

In [10]:
print(final_state['content'])

## Rise of AI: A Comprehensive Look

**I. Introduction (Hook & Context)**

*   **A. Hook:** Did you know AI is projected to contribute a staggering $15.7 trillion to the global economy by 2030? This isn't just hype; it's a testament to the transformative power of Artificial Intelligence, a force already reshaping our world in profound ways.
*   **B. Define Artificial Intelligence:** Artificial Intelligence (AI) refers to the ability of machines to mimic human cognitive functions.  It encompasses a wide range of techniques that allow computers to learn from data, solve problems, and make decisions with minimal human intervention.  We often categorize AI into different levels: **Narrow or Weak AI** (designed for specific tasks, like playing chess), **General or Strong AI** (possessing human-level intelligence across various domains – currently hypothetical), and **Super AI** (surpassing human intelligence – also theoretical).
*   **C. Brief History of AI:** The concept of intelligent mac